## 라이브러리 로드 및 환경변수 로드


In [1]:
import os
from datetime import datetime
from dotenv import load_dotenv
import requests
import pandas as pd

load_dotenv()
# 환경 변수 로드
API_KEY = os.getenv("API_KEY")
tm = datetime.now().strftime("%Y%m%H%M")
stn = 0  # 지상 관측 지점 코드

## API 로 받아온 데이터의 DataFrame으로의 변환 저장

- `class CreateDataFrameFromText`
- `class CreateDataFrameFromJson`


In [2]:
class CreateDataFrameFromText:
    """
    기상청에서 api 추출 후 데이터 파싱
    - 형식이 제대로 되어있지 않아서 공백split()을 통해서 파싱 처리
    - 헤더 정보 추출 시 주석에서 헤더 정보 추출
    - 데이터 파싱 시 예외 케이스 처리
    """

    def __init__(self, url, header=None):
        self.url = url
        self.header = header

    def get_response(self):
        response = requests.get(self.url)
        response = response.text
        return response

    def get_data(self, txt):
        """
        데이터 파싱
        """

        # 예외 케이스
        txt = txt.replace(" Gun", "gun")

        lines = txt.strip().split("\n")
        for idx, line in enumerate(lines):
            if not line.startswith("#"):
                break
        data_rows = lines[idx:-1]
        data = [data_row.strip().split() for data_row in data_rows]
        return data

    def get_header_data(self, txt):
        """주석에서 헤더 정보 추출"""

        headers = []
        lines = txt.strip().split("\n")

        for line in lines:
            if line.startswith("#") and ":" in line:
                header_name = line.split(":")[0].split()[-1]
                headers.append(header_name)

        return headers

    def check_data(self, header, columns):
        print(f"header {header}, len: {len(header)}")
        for i in columns:
            if len(i) != len(header):
                print(f"column len: {i},{len(i)}")
                return False
        return True

    def create_dataframe(self):
        txt = self.get_response()
        data = self.get_data(txt)

        if self.header:
            header = self.header
        else:
            header = self.get_header_data(txt)

        if not self.check_data(header, data):
            raise ValueError("header와 columns의 길이가 다릅니다.")

        df = pd.DataFrame(data, columns=header)
        return df

In [3]:
class CreateDataFrameFromJson:
    """
    기상청에서 api 추출 후 데이터 파싱
    - json 형식으로 추출 후 데이터 파싱
    """

    def __init__(self, url, tm=None):
        self.url = url
        self.tm = tm

    def get_response(self):
        response = requests.get(self.url)
        response = response.json()
        return response["response"]["body"]["items"]["item"]

    def create_dataframe(self):
        data = self.get_response()
        print(len(data))
        df = pd.DataFrame(data)
        # df.insert(0, "TM", self.tm) -> 예보 시각 추가
        return df

#### CreateDataFrameFromText 테스트


In [8]:
urls = {
    "location": f"https://apihub.kma.go.kr/api/typ01/url/stn_inf.php?inf=SFC&tm={tm}&help=1&authKey={API_KEY}",  # 지상 관측 지점 정보
    "time_series": f"https://apihub.kma.go.kr/api/typ01/url/kma_sfctm2.php?tm={tm}&help=1&authKey={API_KEY}",  # 단일 시간 모든 지점
    "time_series_by_stn": f"https://apihub.kma.go.kr/api/typ01/url/kma_sfctm2.php?tm={tm}&stn={stn}&help=1&authKey={API_KEY}",  # 기간 시간 특정 지점
}


time_series_url = urls["location"]
print(time_series_url)

header_location = [
    "STN_ID",
    "LON",
    "LAT",
    "STN_SP",
    "HT",
    "HT_PA",
    "HT_TA",
    "HT_WD",
    "HT_RN",
    "STN_CD",
    "STN_KO",
    "STN_EN",
    "FCT_ID",
    "LAW_ID",
    "BASIN",
]
df = CreateDataFrameFromText(
    time_series_url, header_location
).create_dataframe()
df.to_csv("./../../data/common/location.csv", index=False)

print(df)

https://apihub.kma.go.kr/api/typ01/url/stn_inf.php?inf=SFC&tm=2025101735&help=1&authKey=SlrOsr3ATpCazrK9wM6QMg
header ['STN_ID', 'LON', 'LAT', 'STN_SP', 'HT', 'HT_PA', 'HT_TA', 'HT_WD', 'HT_RN', 'STN_CD', 'STN_KO', 'STN_EN', 'FCT_ID', 'LAW_ID', 'BASIN'], len: 15
   STN_ID           LON          LAT STN_SP      HT   HT_PA HT_TA  HT_WD  \
0      90  128.56473000  38.25085000  35100   17.53   18.73  1.70  10.00   
1      93  127.75443000  37.94738000  31201   95.78   96.78  1.50  10.00   
2      95  127.30420000  38.14787000  31110  155.48  156.98  1.80  13.00   
3      98  127.06070000  37.90188000  22200  115.62  116.74  1.70  10.00   
4      99  126.76648000  37.88589000  22300   30.59   31.99  1.70  10.00   
..    ...           ...          ...    ...     ...     ...   ...    ...   
92    288  128.74412000  35.49147000  76300    8.31   12.50  2.00  10.00   
93    289  127.87910000  35.41300000  76400  138.22  138.80  1.60  10.00   
94    294  128.60459000  34.88818000  76500   44.83  

#### CreateDataFrameFromJson 테스트 및 예보 데이터 취합 & 저장


예보 24시간 동안만 확인 가능한 api 최근 24시간 자료

- 하나의 datetime을 설정하여 url 체크 및 csv 덧붙이기
- https://apihub.kma.go.kr/api/typ02/openApi/MidFcstInfoService/getMidLandFcst


In [ ]:
tm_noon = datetime.now().strftime("202510240600")
print(tm_noon)
# 일 2회(06:00,18:00)회 생성 되며 발표시각을 입력 YYYYMMDD0600(1800)-최근 24시간 자료만 제공

raining_forcast_url = f"https://apihub.kma.go.kr/api/typ02/openApi/MidFcstInfoService/getMidLandFcst?pageNo=1&numOfRows=10&dataType=JSON&regId=11B00000&tmFc={tm_noon}&authKey={API_KEY}"


new_df = CreateDataFrameFromJson(
    raining_forcast_url, tm_noon
).create_dataframe()
filename = "./forecast_rain.csv"

# ---------------------- check csv file
# try:
#     df = pd.read_csv(filename)
#     combined_df = pd.concat([df, new_df], ignore_index=True)
#     combined_df.to_csv(filename, index=False)

# except FileNotFoundError:
#     new_df.to_csv(filename, mode="w", header=True, index=False)

202510240600


예보 지역 구분 api

- 총 데이터 855개
- 모두 페이지 별로 업데이트 해서 FCT_loc.csv 파일로 저장


In [12]:
for idx in range(1, 9):
    FCT_loc_url = f"https://apihub.kma.go.kr/api/typ02/openApi/FcstZoneInfoService/getFcstZoneCd?pageNo={idx}&numOfRows=100&dataType=JSON&authKey={API_KEY}"
    data = CreateDataFrameFromJson(FCT_loc_url).create_dataframe()
    data.to_csv("./FCT_loc.csv", mode="a", header=False, index=False)

100
100
100
100
100
100
100
55


csv 파일 인코딩 문제로 인해 변환

- https://data.kma.go.kr/data/weatherReport/mrfList.do?pgmNo=646
- 중기 예보 데이터 파일셋을 전체로 2025년을 가져와서 저장 후 변환


In [ ]:
forecast_data = pd.read_csv("./FCT_2025.csv", encoding="euc-kr")
forecast_data.to_csv("./FCT_2025.csv", index=False)

중기예보 데이터 셋 확인 및 변환


In [64]:
forecast = pd.read_csv("./FCT_2025.csv")
print(forecast)
print(forecast.columns)

                 발표시각        지역           예보시각       예보  강수확률(%)
0      2025-01-01 06시  서울.인천.경기  2025-01-05 오전     구름많음       20
1      2025-01-01 06시  서울.인천.경기  2025-01-05 오후  흐리고 비/눈       70
2      2025-01-01 06시  서울.인천.경기  2025-01-06 오전  흐리고 비/눈       60
3      2025-01-01 06시  서울.인천.경기  2025-01-06 오후     구름많음       20
4      2025-01-01 06시  서울.인천.경기  2025-01-07 오전     구름많음       20
...               ...       ...            ...      ...      ...
59195  2025-10-23 18시      경상남도  2025-10-30 오전     구름많음       30
59196  2025-10-23 18시      경상남도  2025-10-30 오후     구름많음       30
59197  2025-10-23 18시      경상남도  2025-10-31 오전       흐림       40
59198  2025-10-23 18시      경상남도  2025-11-01 오전       맑음       20
59199  2025-10-23 18시      경상남도  2025-11-02 오전       맑음       10

[59200 rows x 5 columns]
Index(['발표시각', '지역', '예보시각', '예보', '강수확률(%)'], dtype='object')


In [ ]:
print(forecast.dtypes)
print(forecast.shape)
print(forecast.info())
print(forecast.describe())
print(forecast.isnull().sum())
print(forecast.isnull().sum() / len(forecast))

발표시각       object
지역         object
예보시각       object
예보         object
강수확률(%)     int64
dtype: object
(59200, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59200 entries, 0 to 59199
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   발표시각     59200 non-null  object
 1   지역       59200 non-null  object
 2   예보시각     59200 non-null  object
 3   예보       59200 non-null  object
 4   강수확률(%)  59200 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 2.3+ MB
None
            강수확률(%)
count  59200.000000
mean      31.448480
std       18.145816
min       10.000000
25%       20.000000
50%       30.000000
75%       40.000000
max       90.000000
발표시각       0
지역         0
예보시각       0
예보         0
강수확률(%)    0
dtype: int64
발표시각       0.0
지역         0.0
예보시각       0.0
예보         0.0
강수확률(%)    0.0
dtype: float64


지역확인


In [31]:
# 지역 컬럼의 고유값 확인
print("=== 지역별 고유값 ===")
print(forecast["지역"].unique())
print(f"\n총 지역 수: {forecast['지역'].nunique()}")

# 지역별 개수 확인
print("\n=== 지역별 데이터 개수 ===")
print(forecast["지역"].value_counts())

=== 지역별 고유값 ===
['서울.인천.경기' '충청북도' '충청남도' '강원영서' '강원영동' '전북자치도' '전라남도' '제주도' '경상북도' '경상남도']

총 지역 수: 10

=== 지역별 데이터 개수 ===
지역
서울.인천.경기    5920
충청북도        5920
충청남도        5920
강원영서        5920
강원영동        5920
전북자치도       5920
전라남도        5920
제주도         5920
경상북도        5920
경상남도        5920
Name: count, dtype: int64


예보 값 확인


In [33]:
# 예보 컬럼의 고유값 확인
print("=== 예보별 고유값 ===")
print(forecast["예보"].unique())
print(f"\n총 예보 종류 수: {forecast['예보'].nunique()}")

# 예보별 개수 확인
print("\n=== 예보별 데이터 개수 ===")
print(forecast["예보"].value_counts())

=== 예보별 고유값 ===
['구름많음' '흐리고 비/눈' '맑음' '흐림' '구름많고 비/눈' '흐리고 눈' '흐리고 비' '구름많고 눈' '구름많고 비'
 '구름많고 소나기']

총 예보 종류 수: 10

=== 예보별 데이터 개수 ===
예보
구름많음        22970
맑음          16184
흐림          14338
흐리고 비        4519
흐리고 비/눈       721
흐리고 눈         257
구름많고 비/눈       90
구름많고 소나기       59
구름많고 비         45
구름많고 눈         17
Name: count, dtype: int64


In [65]:
# 지역명과 코드 매핑 딕셔너리 생성
region_mapping = {
    "서울.인천.경기": "11B00000",
    "충청북도": "11C10000",
    "충청남도": "11C20000",
    "강원영서": "11D10000",
    "강원영동": "11D20000",
    "전북자치도": "11F10000",
    "전라남도": "11F20000",
    "제주도": "11G00000",
    "경상북도": "11H10000",
    "경상남도": "11H20000",
}

forecast["지역코드"] = forecast["지역"].map(region_mapping)
print(forecast[["지역", "지역코드"]].drop_duplicates().sort_values("지역"))

          지역      지역코드
44      강원영동  11D20000
33      강원영서  11D10000
99      경상남도  11H20000
88      경상북도  11H10000
0   서울.인천.경기  11B00000
66      전라남도  11F20000
55     전북자치도  11F10000
77       제주도  11G00000
22      충청남도  11C20000
11      충청북도  11C10000


In [66]:
print(forecast.head())
forecast.to_csv("./FCT_2025_with_location_code.csv", index=False)

             발표시각        지역           예보시각       예보  강수확률(%)      지역코드
0  2025-01-01 06시  서울.인천.경기  2025-01-05 오전     구름많음       20  11B00000
1  2025-01-01 06시  서울.인천.경기  2025-01-05 오후  흐리고 비/눈       70  11B00000
2  2025-01-01 06시  서울.인천.경기  2025-01-06 오전  흐리고 비/눈       60  11B00000
3  2025-01-01 06시  서울.인천.경기  2025-01-06 오후     구름많음       20  11B00000
4  2025-01-01 06시  서울.인천.경기  2025-01-07 오전     구름많음       20  11B00000


그냥 sorting만 해주는 그룹화. 모아주는 역할만 함


In [61]:
grouped_forecast = (
    forecast.groupby(["예보시각", "지역", "발표시각"])["강수확률(%)"]
    .mean()
    .reset_index()
)

print("=== 그룹화된 데이터 ===")
print(grouped_forecast.loc[170:220])

=== 그룹화된 데이터 ===
              예보시각        지역            발표시각  강수확률(%)
170  2025-01-07 오후      충청남도  2025-01-01 06시     20.0
171  2025-01-07 오후      충청남도  2025-01-01 18시     20.0
172  2025-01-07 오후      충청남도  2025-01-02 06시     20.0
173  2025-01-07 오후      충청남도  2025-01-02 18시     20.0
174  2025-01-07 오후      충청남도  2025-01-03 06시     40.0
175  2025-01-07 오후      충청북도  2025-01-01 06시     20.0
176  2025-01-07 오후      충청북도  2025-01-01 18시     20.0
177  2025-01-07 오후      충청북도  2025-01-02 06시     20.0
178  2025-01-07 오후      충청북도  2025-01-02 18시     20.0
179  2025-01-07 오후      충청북도  2025-01-03 06시     30.0
180  2025-01-08 오전      강원영동  2025-01-01 06시     20.0
181  2025-01-08 오전      강원영동  2025-01-01 18시     10.0
182  2025-01-08 오전      강원영동  2025-01-02 06시     10.0
183  2025-01-08 오전      강원영동  2025-01-02 18시     10.0
184  2025-01-08 오전      강원영동  2025-01-03 06시     10.0
185  2025-01-08 오전      강원영동  2025-01-03 18시     10.0
186  2025-01-08 오전      강원영동  2025-01-04 06시     10.0
187  2025-0

4일전 - 7일전에 예측한 값에 대한 평균을 구하는 그룹화


In [ ]:
grouped_forecast = (
    forecast.groupby(["예보시각", "지역"])[
        "강수확률(%)"
    ]  # 4일전 - 7일전에 예측한 값에 대한 평균
    .mean()
    .reset_index()
)

print("=== 그룹화된 데이터 ===")
print(grouped_forecast.loc[170:220])

=== 그룹화된 데이터 ===
              예보시각        지역    강수확률(%)
170  2025-01-13 오후      강원영동  10.000000
171  2025-01-13 오후      강원영서  21.428571
172  2025-01-13 오후      경상남도  10.000000
173  2025-01-13 오후      경상북도  10.000000
174  2025-01-13 오후  서울.인천.경기  17.142857
175  2025-01-13 오후      전라남도  21.428571
176  2025-01-13 오후     전북자치도  20.000000
177  2025-01-13 오후       제주도  31.428571
178  2025-01-13 오후      충청남도  18.571429
179  2025-01-13 오후      충청북도  18.571429
180  2025-01-14 오전      강원영동  13.846154
181  2025-01-14 오전      강원영서  34.615385
182  2025-01-14 오전      경상남도  10.769231
183  2025-01-14 오전      경상북도  13.846154
184  2025-01-14 오전  서울.인천.경기  18.461538
185  2025-01-14 오전      전라남도  26.923077
186  2025-01-14 오전     전북자치도  28.461538
187  2025-01-14 오전       제주도  26.923077
188  2025-01-14 오전      충청남도  23.076923
189  2025-01-14 오전      충청북도  19.230769
190  2025-01-14 오후      강원영동  10.000000
191  2025-01-14 오후      강원영서  20.000000
192  2025-01-14 오후      경상남도  25.714286
193  2025-01-14 오후     

In [52]:
# 발표시각을 날짜와 시간으로 분리해서 더 세밀하게 분석
forecast["발표날짜"] = pd.to_datetime(
    forecast["발표시각"].str.replace("시", ""), format="%Y-%m-%d %H"
).dt.date
forecast["발표시간"] = pd.to_datetime(
    forecast["발표시각"].str.replace("시", ""), format="%Y-%m-%d %H"
).dt.hour

# 예보시각, 지역, 발표날짜로 그룹화
grouped_by_date = (
    forecast.groupby(["예보시각", "지역", "발표날짜"])["강수확률(%)"]
    .mean()
    .reset_index()
)

print("=== 날짜별 그룹화 ===")
print(grouped_by_date.head())

=== 날짜별 그룹화 ===
            예보시각        지역        발표날짜  강수확률(%)
0  2025-01-05 오전      강원영동  2025-01-01     20.0
1  2025-01-05 오전      강원영서  2025-01-01     40.0
2  2025-01-05 오전      경상남도  2025-01-01     20.0
3  2025-01-05 오전      경상북도  2025-01-01     30.0
4  2025-01-05 오전  서울.인천.경기  2025-01-01     20.0


날짜에 대한 전처리


In [69]:
df_new_forecast = forecast.copy()
df_new_forecast.columns = [
    "forecast_time",
    "region",
    "forecast_issue_time",
    "weather_condition",
    "rain_probability",
    "region_code",
]

In [70]:
print(df_new_forecast.head())

    forecast_time    region forecast_issue_time weather_condition  \
0  2025-01-01 06시  서울.인천.경기       2025-01-05 오전              구름많음   
1  2025-01-01 06시  서울.인천.경기       2025-01-05 오후           흐리고 비/눈   
2  2025-01-01 06시  서울.인천.경기       2025-01-06 오전           흐리고 비/눈   
3  2025-01-01 06시  서울.인천.경기       2025-01-06 오후              구름많음   
4  2025-01-01 06시  서울.인천.경기       2025-01-07 오전              구름많음   

   rain_probability region_code  
0                20    11B00000  
1                70    11B00000  
2                60    11B00000  
3                20    11B00000  
4                20    11B00000  


In [ ]:
# 1. forecast_time (발표시각) 변환: "2025-01-01 06시" -> datetime
df_new_forecast["forecast_time"] = pd.to_datetime(
    df_new_forecast["forecast_time"].str.replace("시", ""),
    format="%Y-%m-%d %H",
)

print(df_new_forecast.head())

        forecast_time    region forecast_issue_time weather_condition  \
0 2025-01-01 06:00:00  서울.인천.경기       2025-01-05 오전              구름많음   
1 2025-01-01 06:00:00  서울.인천.경기       2025-01-05 오후           흐리고 비/눈   
2 2025-01-01 06:00:00  서울.인천.경기       2025-01-06 오전           흐리고 비/눈   
3 2025-01-01 06:00:00  서울.인천.경기       2025-01-06 오후              구름많음   
4 2025-01-01 06:00:00  서울.인천.경기       2025-01-07 오전              구름많음   

   rain_probability region_code  
0                20    11B00000  
1                70    11B00000  
2                60    11B00000  
3                20    11B00000  
4                20    11B00000  


In [72]:
# 2. forecast_issue_time (예보시각) 변환: "2025-01-05 오전" -> datetime
def convert_forecast_issue_time(time_str):
    # "2025-01-05 오전" -> "2025-01-05 06:00"
    # "2025-01-05 오후" -> "2025-01-05 18:00"
    date_part = time_str.split(" ")[0]
    time_part = time_str.split(" ")[1]

    if time_part == "오전":
        return pd.to_datetime(f"{date_part} 06:00")
    elif time_part == "오후":
        return pd.to_datetime(f"{date_part} 18:00")
    else:
        return pd.to_datetime(f"{date_part} 12:00")


df_new_forecast["forecast_issue_time"] = df_new_forecast[
    "forecast_issue_time"
].apply(convert_forecast_issue_time)

print(df_new_forecast.head())

# 3. 강수확률을 숫자로 변환
df_new_forecast["rain_probability"] = pd.to_numeric(
    df_new_forecast["rain_probability"], errors="coerce"
)

        forecast_time    region forecast_issue_time weather_condition  \
0 2025-01-01 06:00:00  서울.인천.경기 2025-01-05 06:00:00              구름많음   
1 2025-01-01 06:00:00  서울.인천.경기 2025-01-05 18:00:00           흐리고 비/눈   
2 2025-01-01 06:00:00  서울.인천.경기 2025-01-06 06:00:00           흐리고 비/눈   
3 2025-01-01 06:00:00  서울.인천.경기 2025-01-06 18:00:00              구름많음   
4 2025-01-01 06:00:00  서울.인천.경기 2025-01-07 06:00:00              구름많음   

   rain_probability region_code  
0                20    11B00000  
1                70    11B00000  
2                60    11B00000  
3                20    11B00000  
4                20    11B00000  


In [75]:
df_new_forecast.to_csv("./../../data/rainfall/forecast.csv", index=False)

기상청 api 허브에서 일별 데이터 가져오기


In [5]:
tm1 = "20250101"
tm2 = "20251028"

date_url = "https://apihub.kma.go.kr/api/typ01/url/kma_sfcdd3.php?tm1=20250101&tm2=20251028&stn=0&help=1&authKey=SlrOsr3ATpCazrK9wM6QMg&mode=0"

print(date_url)

df_date = CreateDataFrameFromText(date_url).create_dataframe()
print(df_date.head())

https://apihub.kma.go.kr/api/typ01/url/kma_sfcdd3.php?tm1=20250101&tm2=20251028&stn=0&help=1&authKey=SlrOsr3ATpCazrK9wM6QMg&mode=0
header ['TM', 'STN', 'WS_AVG', 'WR_DAY', 'WD_MAX', 'WS_MAX', 'WS_MAX_TM', 'WD_INS', 'WS_INS', 'WS_INS_TM', 'TA_AVG', 'TA_MAX', 'TA_MAX_TM', 'TA_MIN', 'TA_MIN_TM', 'TD_AVG', 'TS_AVG', 'TG_MIN', 'HM_AVG', 'HM_MIN', 'HM_MIN_TM', 'PV_AVG', 'EV_S', 'EV_L', 'FG_DUR', 'PA_AVG', 'PS_AVG', 'PS_MAX', 'PS_MAX_TM', 'PS_MIN', 'PS_MIN_TM', 'CA_TOT', 'SS_DAY', 'SS_DUR', 'SS_CMB', 'SI_DAY', 'SI_60M_MAX', 'SI_60M_MAX_TM', 'RN_DAY', 'RN_D99', 'RN_DUR', 'RN_60M_MAX', 'RN_60M_MAX_TM', 'RN_10M_MAX', 'RN_10M_MAX_TM', 'RN_POW_MAX', 'RN_POW_MAX_TM', 'SD_NEW', 'SD_NEW_TM', 'SD_MAX', 'SD_MAX_TM', 'TE_05', 'TE_10', 'TE_15', 'TE_30', 'TE_50'], len: 56
         TM STN WS_AVG WR_DAY WD_MAX WS_MAX WS_MAX_TM WD_INS WS_INS WS_INS_TM  \
0  20250101  90    3.3   2866     29    6.0        20     27   11.6      1025   
1  20250101  93    0.8    717     23    4.0      1547     23    6.6      15

In [6]:
df_date.to_csv("./rainfall_daily.csv", index=False)

In [15]:
headers = ["TM", "STN", "RN_DUR", "RN_D99"]

df_rainfall = df_date[headers].copy()
print(df_rainfall.head())

         TM STN RN_DUR RN_D99
0  20250101  90  -9.00   -9.0
1  20250101  93  -9.00   -9.0
2  20250101  95  -9.00   -9.0
3  20250101  98  -9.00   -9.0
4  20250101  99  -9.00   -9.0


In [16]:
df_rainfall["RN_DUR"] = pd.to_numeric(df_rainfall["RN_DUR"], errors="coerce")
df_rainfall["RN_D99"] = pd.to_numeric(df_rainfall["RN_D99"], errors="coerce")

In [20]:
df_rainfall["RAIN_YN"] = (df_rainfall["RN_DUR"] > 0) | (
    df_rainfall["RN_D99"] > 0
)
print(df_rainfall.head(50))

columns = ["date", "stn_id", "rain_dur", "rain_d99", "rain_yn"]
df_rainfall.columns = columns
print(df_rainfall.head(50))
df_rainfall.to_csv("./../../data/rainfall/rainfall_daily.csv", index=False)

          TM  STN  RN_DUR  RN_D99  RAIN_YN
0   20250101   90    -9.0    -9.0    False
1   20250101   93    -9.0    -9.0    False
2   20250101   95    -9.0    -9.0    False
3   20250101   98    -9.0    -9.0    False
4   20250101   99    -9.0    -9.0    False
5   20250101  100    -9.0    -9.0    False
6   20250101  101    -9.0    -9.0    False
7   20250101  102    -9.0    -9.0    False
8   20250101  104    -9.0    -9.0    False
9   20250101  105    -9.0    -9.0    False
10  20250101  106    -9.0    -9.0    False
11  20250101  108    -9.0    -9.0    False
12  20250101  112    -9.0    -9.0    False
13  20250101  114    -9.0    -9.0    False
14  20250101  115    -9.0    -9.0    False
15  20250101  119    -9.0    -9.0    False
16  20250101  121    -9.0    -9.0    False
17  20250101  127    -9.0    -9.0    False
18  20250101  129    -9.0    -9.0    False
19  20250101  130    -9.0    -9.0    False
20  20250101  131    -9.0    -9.0    False
21  20250101  133    -9.0    -9.0    False
22  2025010